# Importando Bibliotecas

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.cluster.hierarchy as sch
import scipy.stats as stats
from scipy.stats import zscore
from scipy.spatial.distance import pdist
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import pingouin as pg
import plotly.express as px 
import plotly.io as pio
import imageio
pio.renderers.default='browser'

# Tratamento de Dados

## Banco de Dados Original - base

In [8]:
base = pd.read_excel('BANCODEDADOS.xlsx')
display(base)

,Customer ID,Customer Name,Customer Type,Customer Value,Customer Potential,Sales Target Midrange
0,22011858,NEXTROM COMERCIO DE MAQUINAS LTDA,OEM,Smart,6.000000e+05,NaN
1,22011866,WHIRLPOOL SA,End User,Smart,5.000000e+05,NaN
2,22011890,SCANIA LATIN AMERICA LTDA,End User,Strategic,2.000000e+06,1543500.0
3,22011890,SCANIA LATIN AMERICA LTDA,Not assigned,Not assigned,NaN,NaN
4,22011908,TTB INDUSTRIA E COMERCIO DE PRODUTO,End User,Smart,1.210000e+04,NaN
...,...,...,...,...,...,...
5943,22159525,SERVICO NACIONAL DE APRENDIZAGEM,Not assigned,Not assigned,NaN,NaN
5944,22159545,FRR INDUSTRIA TECNICA DE CONVERSAO,Not assigned,Smart,2.000000e+04,NaN
5945,22159563,LSI BRASIL LTDA,End User,Strategic,NaN,NaN
5946,#,Not assigned,Not assigned,Not assigned,NaN,NaN


## Remover linhas que contêm 'NaNs'

In [9]:
# Remover linhas que contêm NaNs

base = base_sem_nan_linhas = base.dropna()

display(base)

,Customer ID,Customer Name,Customer Type,Customer Value,Customer Potential,Sales Target Midrange
2,22011890,SCANIA LATIN AMERICA LTDA,End User,Strategic,2000000.0,1543500.0
5,22011932,INDUSTRIAS ARTEB S/A,End User,Smart,100000.0,60000.0
6,22012500,TENNECO AUTOMOTIVE BRASIL LTDA,End User,Strategic,300000.0,99225.0
7,22012583,PRENSAS SCHULER S A,OEM,Strategic,650000.0,441000.0
8,22013276,HAVER & BOECKER LATINOAMERICANA MAQ,OEM,Strategic,2350000.0,2300000.0
...,...,...,...,...,...,...
5932,22159302,NILGRAS MAQUINAS E SOLUCOES INDUSTR,OEM,Important,250000.0,150000.0
5933,22159309,DELIVIO & PETENUCCI MAQUINAS,Distributor,Smart,200000.0,80000.0
5937,22159426,MP SERVICE IMP. EXP AUTOMAÇÃO,OEM,Smart,250000.0,150000.0
5938,22159436,SMACMOTION MAQUINAS E EQUIPAMENTOS,OEM,Important,210000.0,125000.0


## Remover linhas que  contêm 'Not assigned'

In [10]:
# Remover linhas onde 'Not assigned' está presente em qualquer uma das colunas

base = base[(base['Customer Type'] != 'Not assigned') & (base['Customer Value'] != 'Not assigned')]

display(base)

,Customer ID,Customer Name,Customer Type,Customer Value,Customer Potential,Sales Target Midrange
2,22011890,SCANIA LATIN AMERICA LTDA,End User,Strategic,2000000.0,1543500.0
5,22011932,INDUSTRIAS ARTEB S/A,End User,Smart,100000.0,60000.0
6,22012500,TENNECO AUTOMOTIVE BRASIL LTDA,End User,Strategic,300000.0,99225.0
7,22012583,PRENSAS SCHULER S A,OEM,Strategic,650000.0,441000.0
8,22013276,HAVER & BOECKER LATINOAMERICANA MAQ,OEM,Strategic,2350000.0,2300000.0
...,...,...,...,...,...,...
5932,22159302,NILGRAS MAQUINAS E SOLUCOES INDUSTR,OEM,Important,250000.0,150000.0
5933,22159309,DELIVIO & PETENUCCI MAQUINAS,Distributor,Smart,200000.0,80000.0
5937,22159426,MP SERVICE IMP. EXP AUTOMAÇÃO,OEM,Smart,250000.0,150000.0
5938,22159436,SMACMOTION MAQUINAS E EQUIPAMENTOS,OEM,Important,210000.0,125000.0


## Análise para verificar se será ou não necessário uma aplicação de Z-Score para parametrizar todas as variáveis

In [11]:
# Obtendo as estatísticas descritivas das variáveis

tab_descritivas = base.describe().T
# Vamos padronizar as variáveis antes da clusterização!

display(tab_descritivas)

,count,mean,std,min,25%,50%,75%,max
Customer Potential,3197.0,417451.405067,874443.000418,2.0,100000.0,200000.0,400000.0,17004396.0
Sales Target Midrange,3197.0,170111.348764,334057.875511,1.0,30000.0,70000.0,160000.0,4000000.0


In [12]:
# Selecionar apenas as colunas numéricas para aplicar o Z-Score
base_numeric = base.select_dtypes(include=[np.number])

# Aplicar Z-Score às colunas numéricas
base_pad = base_numeric.apply(zscore, ddof=1)

# Visualizar o resultado do procedimento na média e desvio padrão
print(round(base_pad.mean(), 3))
print(round(base_pad.std(), 3))
print(base_pad)

Customer Potential       0.0
Sales Target Midrange    0.0
dtype: float64
Customer Potential       1.0
Sales Target Midrange    1.0
dtype: float64
      Customer Potential  Sales Target Midrange
2               1.809779               4.111230
5              -0.363033              -0.329618
6              -0.134316              -0.212198
7               0.265939               0.810903
8               2.210034               6.375807
...                  ...                    ...
5932           -0.191495              -0.060203
5933           -0.248674              -0.269748
5937           -0.191495              -0.060203
5938           -0.237238              -0.135041
5941           -0.134316              -0.299683

[3197 rows x 2 columns]


## Selecionando apenas varíaveis métricas para iniciar a Clusterização

In [13]:
base_metrica = base.drop(columns=['Customer ID', 'Customer Name', 'Customer Type', 'Customer Value'])
display(base_metrica)

,Customer Potential,Sales Target Midrange
2,2000000.0,1543500.0
5,100000.0,60000.0
6,300000.0,99225.0
7,650000.0,441000.0
8,2350000.0,2300000.0
...,...,...
5932,250000.0,150000.0
5933,200000.0,80000.0
5937,250000.0,150000.0
5938,210000.0,125000.0


# Início da Análise de Cluster -  Método não Hierárquio K-means

In [14]:
# único parãmetro que informamos é o n_clusters=3 (no nosso caso  escolhemos 3 como imput)

kmeans_final = KMeans(n_clusters=3, init='random', random_state=100).fit(base_pad)

kmeans_clusters = kmeans_final.labels_

base_metrica['cluster_kmeans'] = kmeans_clusters
base_pad['cluster_kmeans'] = kmeans_clusters
base_metrica['cluster_kmeans'] = base_metrica['cluster_kmeans'].astype('category')
base_pad['cluster_kmeans'] = base['cluster_kmeans'].astype('category')
display(base)

C:\Users\NOTE MATHEUS\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



KeyError: 'cluster_kmeans'

# Teste Estatístico F - verificar quais variáveis auxiliaram na formação dos Clusters

##  Análise de variância de um fator (ANOVA)

In [15]:
# Customer Potential

pg.anova(dv='Customer Potential', 
         between='cluster_kmeans', 
         data=base_pad,
         detailed=True).T

,0,1
Source,cluster_kmeans,Within
SS,2311.419648,884.580352
DF,2,3194
MS,1155.709824,0.276951
F,4172.981199,NaN
p-unc,0.0,NaN
np2,0.723223,NaN


P-valor = 0

P-valor < 5% - Logo é estatisticamente siginificativa para formação de pelo menos um dos clusters

In [16]:
# Sales Target Midrange

pg.anova(dv='Sales Target Midrange', 
         between='cluster_kmeans', 
         data=base_pad,
         detailed=True).T

,0,1
Source,cluster_kmeans,Within
SS,2416.090617,779.909383
DF,2,3194
MS,1208.045308,0.24418
F,4947.365423,NaN
p-unc,0.0,NaN
np2,0.755973,NaN


P-valor = 0

P-valor < 5% - Logo é estatisticamente siginificativa para formação de pelo menos um dos clusters

Todas as variáveis foram significativas pra formação de pelo menos um dos clusters.

In [17]:
#%% Gráfico 3D dos clusters

fig = px.scatter(base_metrica, 
                    x='Sales Target Midrange', 
                    y='Customer Potential',
                    color='cluster_kmeans')
fig.show()

In [18]:
#%% Identificação das características dos clusters

# Agrupando o banco de dados

base_grupo = base_metrica.groupby(by=['cluster_kmeans'])

# Estatísticas descritivas por grupo

tab_desc_grupo = base_grupo.describe().T

display(tab_desc_grupo)

#%% FIM

cluster_kmeans                          0             1             2
Customer Potential    count  2.978000e+03  2.500000e+01  1.940000e+02
                      mean   2.612385e+05  7.545307e+06  1.896862e+06
                      std    2.630383e+05  3.360972e+06  1.016220e+06
                      min    2.000000e+00  3.030000e+06  3.000000e+04
                      25%    9.680000e+04  5.361693e+06  1.150000e+06
                      50%    1.800000e+05  6.475000e+06  1.650000e+06
                      75%    3.500000e+05  9.067970e+06  2.487500e+06
                      max    2.500000e+06  1.700440e+07  5.800000e+06
Sales Target Midrange count  2.978000e+03  2.500000e+01  1.940000e+02
                      mean   1.015779e+05  2.624748e+06  9.058157e+05
                      std    1.090694e+05  6.908788e+05  4.561805e+05
                      min    1.000000e+00  1.200000e+06  1.000000e+05
                      25%    3.000000e+04  2.326000e+06  6.000000e+05
                      50%    6.000000e+04  2.600000e+06  8.000000e+05
                      75%    1.367500e+05  3.000000e+06  1.049344e+06
                      max    6.289190e+05  4.000000e+06  2.650000e+06